In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob


import mysql.connector as mysql
from sqlalchemy import create_engine as dbengine


In [8]:
def create_dbconnection():
    try:
        db = mysql.connect(host=os.getenv('MYSQL_SERVER'),
                            database=os.getenv('MYSQL_DATABASE'),
                            user=os.getenv('MYSQL_USER'),
                            password=os.getenv('MYSQL_PASSWORD'))
        return db, db.cursor()
    except mysql.Error as e:
        if e.errno == sqlError.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
            return None
        elif e.errno == sqlError.ER_BAD_DB_ERROR:
            print("Database does not exist")
            return None
        else:
            print(err)
            return None
    else:
        return None

In [9]:
def execute_dbquery(query, db=None, cursor=None):
    # If no db or cursor is provided, connect to the database
    if db is None or cursor is None:
        db, cursor = connect_to_db()

    try:
        cursor.execute(query)
        db.commit()
    except mysql.Error as e:
        print(f"Failed creating database with query: {query} - Error: {err}")
        exit(1)


In [ ]:
### For each tableName, drop table from the database
# for tableName in tableNames:
#     createQuery = f'DROP TABLE IF EXISTS {tableName}'
#     cursor.execute(createQuery)

In [10]:
db, cursor = create_dbconnection()

In [66]:
### Read the dictionary file
dbDictionary = pd.read_excel('@dictionary.xlsx', sheet_name='Tables21')
dbValues = pd.read_excel('@dictionary.xlsx', sheet_name='valuesets21')
### Create a list of tuples with the table names and the data files
tableNames = list(zip(dbDictionary.TIM7020TableName, dbDictionary.TableName))

for myTableName, ipedsTableName in tableNames:
    ### Create the table
    execute_dbquery(f'CREATE TABLE IF NOT EXISTS {myTableName} (ipeds_id INTEGER);', db=db, cursor=cursor)
    ### Read the dictionary file
    print(f'myTableName: {myTableName}, ipedsTableName: {ipedsTableName}')
    ### Read table's dictionary file
    tableDictionary = pd.read_excel(f'dictionary/{ipedsTableName}.xlsx', sheet_name=1)
    ### Read first 100 rows of the data file
    tableData = pd.read_csv(f'data/{ipedsTableName}.csv', nrows=100, encoding="utf-8", na_values=['.', ' '])
    ### Create string for the query
    query = f'ALTER TABLE {myTableName} '
    ### Loop through the rows of the table dictionary file
    for _, row in tableDictionary.iterrows():
        ## Skip the IPEDS ID
        if row.varnumber == 1:
            continue
        ## If there is an imputation variable, add it to the table
        if (isinstance(row.imputationvar, str) == True) and (row.imputationvar != 'None'):
            # print(row.varnumber, row.varname, row.DataType, row.Fieldwidth, row.imputationvar, row.varTitle)
            query += f'ADD COLUMN IF NOT EXISTS {row.imputationvar} CHAR(1), '
        ## If the column is an alpha column, add it to the table
        dataType = 'TEXT'
        if (row.format == 'Alpha'):
            dataType = f'VARCHAR({row.Fieldwidth})'
        elif (row.format == 'Cont'):
            ## Check to see if data type is integer or float
            if tableData[row.varname.upper()].dtype == 'float':
                if row.Fieldwidth > 8:
                    dataType = 'DOUBLE'
                else:
                    dataType = 'FLOAT'
            else:
                if row.Fieldwidth > 8:
                    dataType = 'BIGINT'
                elif row.Fieldwidth > 4:
                    dataType = 'INT'
                elif row.Fieldwidth == 1:
                    dataType = 'TINYINT'
            if row.DataType == 'float':
                dataType = 'REAL'
        elif (row.format == 'Disc'):
            dataType = 'ENUM()'
        
        addColumn = f'ADD COLUMN IF NOT EXISTS {row.varname} {dataType} '
        print(addColumn)
        # query += addColumn
        # ### Create the column
        # print(row["VARNAME"], row["TYPE"])
        # query = f'ALTER TABLE {myTableName} ADD COLUMN IF NOT EXISTS {row["VARNAME"]} {row["TYPE"]};'
        # execute_dbquery(query=query, db=db, cursor=cursor)


myTableName: inst_ic_directory, ipedsTableName: HD2021
varname: INSTNM, DataType: A, Fieldwidth: 120, imputationvar: nan, format: Alpha, dataType: VARCHAR(120)
varname: IALIAS, DataType: A, Fieldwidth: 2000, imputationvar: nan, format: Alpha, dataType: VARCHAR(2000)
varname: ADDR, DataType: A, Fieldwidth: 100, imputationvar: nan, format: Alpha, dataType: VARCHAR(100)
varname: CITY, DataType: A, Fieldwidth: 30, imputationvar: nan, format: Alpha, dataType: VARCHAR(30)
varname: STABBR, DataType: A, Fieldwidth: 2, imputationvar: nan, format: Disc, dataType: ENUM()
varname: ZIP, DataType: A, Fieldwidth: 10, imputationvar: nan, format: Alpha, dataType: VARCHAR(10)
varname: FIPS, DataType: N, Fieldwidth: 2, imputationvar: nan, format: Disc, dataType: ENUM()
varname: OBEREG, DataType: N, Fieldwidth: 2, imputationvar: nan, format: Disc, dataType: ENUM()
varname: CHFNM, DataType: A, Fieldwidth: 50, imputationvar: nan, format: Alpha, dataType: VARCHAR(50)
varname: CHFTITLE, DataType: A, Fieldwidt

In [ ]:
# import openai

# openai.api_key = os.getenv("OPENAI_API_KEY")

# response = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo",
#   messages=[],
#   temperature=0,
#   max_tokens=1024
# )